# Meshing a protein using PyGAMer

This tutorial will walk you through the process of using PyGAMer to generate a tetrahedral mesh of the volume surrounding a protein. The protein of interest, PDBID 2JHO, is that of sperm whale myoglobin at 1.4Å resolution.

In [1]:
# Load in PyGAMer and numpy
import pygamer
import numpy as np
# We will use threevis to visualize the mesh in the notebook
import threevis

In [2]:
pygamer.__version__()

'v2.0.2-dev-16-g36333b7'

In [3]:
# Mesh the protein of interest
mesh = pygamer.readPDB_molsurf('../data/2jho.pdb')

When using PyGAMer it's important to initialize the orientation of the simplices. Notably `compute_orientation()` will try to apply a self consistent set of orientations. In many cases, you may wish to ensure that the mesh normals are outward facing. This can be achieved by calling `correctNormals()` which essentially computes the volume of the bounded surface and flips the normals if the volume is negative.

In [4]:
# Compute the normal orientation
components, orientable, manifold = mesh.compute_orientation()
mesh.correctNormals()
print(F"The mesh has {components} components, is"
      F" {'orientable' if orientable else 'non-orientable'}, and is" 
      F" {'manifold' if manifold else 'non-manifold'}.")

The mesh has 2 components, is orientable, and is manifold.


Note that this mesh has two components. This is because there is some empty space inside of the protein which however is not solvent acessible. Therefore when the surface is extracted, this void space is encapsulated by a separate mesh. You can split the two surfaces accordingly as follows.

In [5]:
meshes = mesh.splitSurfaces()
print(meshes[0].getVolume()*1e9)

for i, m in enumerate(meshes):
    print(F"Mesh {i} is {m.getVolume()} in volume.")

mesh = meshes[0]

19248321147885.734
Mesh 0 is 19248.321147885734 in volume.
Mesh 1 is 51.60539976197597 in volume.


TypeError: 'pygamer.surfacemesh.SurfaceMesh' object does not support indexing

In [ ]:
protverts, protedges, protfaces = meshes[0].to_ndarray()
bverts, bedges, bfaces = meshes[1].to_ndarray()
ctx = threevis.Context(width=640, height=480)
colors = np.zeros((len(bfaces), 3))
colors[:,0] = np.ones(len(bfaces))
ctx.draw_faces(bverts, bfaces, colors=threevis.FaceAttribute(colors))
ctx.draw_edges(protverts, protedges)
ctx.display()

## Conditioning the protein mesh

In [ ]:
# Set selection of all vertices to True so smooth will operate on them.
for v in mesh.vertexIDs:
    v.data().selected = True
# Apply 5 iterations of smoothing
mesh.smooth(max_iter=5, preserve_ridges=True, verbose=True)

In [ ]:
print(F"The mesh currently has {mesh.nVertices} vertices, \
{mesh.nEdges} edges, and {mesh.nFaces} faces.")

That's a lot of vertices! Here's what the protein mesh looks like at this point. `threevis` provides a convenient way to visualize meshes inside of a jupyter-notebook. It uses arrays of vertex positions, edge and face vertex indices. These arrays can be generated using the `to_ndarray()` function.

In [ ]:
protverts, protedges, protfaces = mesh.to_ndarray()
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(protverts, protfaces)
ctx.draw_edges(protverts, protedges)
ctx.display()

### Iterative Coarsening

Depending on your problem, you may wish to coarsen or decimate the mesh. Basically reduce the number of vertices. GAMer offers several functions to help you with this goal.

In [ ]:
for i in range(25):
    # Coarsen dense regions of the mesh
    mesh.coarse_dense(rate = 3, numiter = 1)
    # Coarsen flat regions of the mesh
    mesh.coarse_flat(rate = 0.1, numiter = 1)
    mesh.smooth(max_iter = 3, preserve_ridges = True, verbose = False)
    print(F"Iteration {i}: {mesh.nVertices} vertices, \
{mesh.nEdges} edges, and {mesh.nFaces} faces.")

In [ ]:
# Smooth the mesh again.
mesh.smooth(max_iter = 10, preserve_ridges = True, verbose = True)

In [ ]:
pygamer.writeOFF('test.off', mesh)

In [ ]:
# Set boundary markers of the mesh to 23
for faceID in mesh.faceIDs:
    faceID.data().marker = 23

In [ ]:
# Get the root metadata
gInfo = mesh.getRoot()
gInfo.ishole = True    # Don't mesh the inside of 
gInfo.marker = -1

In [ ]:
# Center mesh at 0,0,0
center, radius = mesh.getCenterRadius()
mesh.translate(-center)

## Visualizing the mesh with threevis

In [ ]:
# Convert mesh to numpy arrays for visualization using threevis
import threevis

protverts, protedges, protfaces = mesh.to_ndarray()
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(protverts, protfaces)
ctx.draw_edges(protverts, protedges)
ctx.display()

## Now let's construct a bounding box to represent the cytosol around the protein

In [ ]:
# Generate a surrounding box
box = pygamer.surfacemesh.cube(5)

In [ ]:
# Set box boundary markers to 50
for faceID in box.faceIDs:
    faceID.data().marker = 50

In [ ]:
# Get and set box metadata
gInfo = box.getRoot()
gInfo.ishole = False
gInfo.marker = 5

In [ ]:
# Scale the box by 2 times the radius of the protein mesh
box.scale(radius*2)

In [ ]:
boxverts, boxedges, boxfaces = box.to_ndarray()
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(boxverts, boxfaces)
ctx.draw_edges(boxverts, boxedges)
# Finally display it
ctx.display()

In [ ]:
# Draw the protein and surrounding box together
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(protverts, protfaces)
ctx.draw_edges(boxverts, boxedges)
ctx.display()

Write the meshes to file...
`pygamer.writeOFF('2jho.off', mesh)`
`pygamer.writeOFF('box.off', box)`

## Tetrahedralizing

In [ ]:
# Construct a list of meshes to pass into TetGen
meshes = [mesh, box]

In [ ]:
# Call tetgen to tetrahedralize. The string represents command line arguments passed directly to TetGen.
tetmesh = pygamer.makeTetMesh(meshes, "q1.3/10O8/7AVC")

In [ ]:
# pygamer.writeDolfin('outputTetMesh.xml', tetmesh)

In [ ]:
toRemove = []
for vertexID in tetmesh.vertexIDs:
    if vertexID.data()[1] > 0.1:
        toRemove.append(vertexID)
for v in toRemove:
    tetmesh.removeVertex(v)
        
surfmesh = tetmesh.extractSurface()
# toRemove = []
# for edgeID in surfmesh.edgeIDs:
#     if len(surfmesh.getCover(edgeID)) < 2:
#         toRemove.append(edgeID)
# for e in toRemove:
#     surfmesh.removeEdge(e)
surfmesh.correctNormals()

v, e, f = surfmesh.to_ndarray()

In [ ]:
# Draw the protein and surrounding box together
ctx = threevis.Context(width=640, height=480)

rgb = np.ones((len(f), 3))

for i, face in enumerate(surfmesh.faceIDs):
    if face.data().marker == 23:
        rgb[i,0] = 1
        rgb[i,1] = 0
        rgb[i,2] = 0
    elif face.data().marker == 50:
        rgb[i,0] = 0
        rgb[i,1] = 0
        rgb[i,2] = 1      
        
colors = threevis.FaceAttribute(rgb)

ctx.draw_faces(v, f, colors=colors)
ctx.draw_edges(v, e)
ctx.display()